<a href="https://colab.research.google.com/github/bei-github/Natural-Language-Processing-in-TensorFlow/blob/master/imdb_Subwords_Multi_Layer_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multiple Layer LSTM

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


In [0]:
tokenizer = info.features['text'].encoder

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),  #8185*64
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),  #return_sequences=True is required for multi layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),  #65*4*64=16640?
    tf.keras.layers.Dense(64, activation='relu'),  #64*64+64
    tf.keras.layers.Dense(1, activation='sigmoid') #64*1+1
])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 312s 798ms/step - loss: 0.5718 - accuracy: 0.6949 - val_loss: 0.4970 - val_accuracy: 0.7884
Epoch 2/10
391/391 [==============================] - 316s 808ms/step - loss: 0.4094 - accuracy: 0.8257 - val_loss: 0.4877 - val_accuracy: 0.7832
Epoch 3/10
391/391 [==============================] - 312s 798ms/step - loss: 0.3440 - accuracy: 0.8624 - val_loss: 0.4717 - val_accuracy: 0.7956
Epoch 4/10
391/391 [==============================] - 308s 788ms/step - loss: 0.2835 - accuracy: 0.8905 - val_loss: 0.6454 - val_accuracy: 0.7762
Epoch 5/10
391/391 [==============================] - 314s 803ms/step - loss: 0.2253 - accuracy: 0.9158 - val_loss: 0.4762 - val_accuracy: 0.8154
Epoch 6/10
391/391 [==============================] - 310s 793ms/step - loss: 0.1894 - accuracy: 0.9299 - val_loss: 0.5302 - val_accuracy: 0.8200
Epoch 7/10
288/391 [=====================>........] - ETA: 58s - loss: 0.1692 - accuracy: 0.9381

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')